# 03 - Modeling & Evaluation

This notebook covers:
- Building baseline and advanced models
- Stratified K-Fold validation
- Evaluation metrics: Accuracy, Precision, Recall, F1, ROC-AUC
- Leaderboard comparison

In [1]:
import pandas as pd
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
import numpy as np

## Load Data

In [2]:
df = pd.read_csv('../data/fe_train.csv')
X = df.drop(['y', 'id'], axis=1)
y = df['y']

In [3]:
X

,age,education,balance,day,duration,campaign,pdays,previous,job_blue-collar,job_entrepreneur,...,month_mar,month_may,month_nov,month_oct,month_sep,poutcome_other,poutcome_success,poutcome_unknown,balance_duration,age_education
0,0.106310,2,-0.422083,25.0,-0.510829,0.155597,-0.302803,-0.223475,False,False,...,False,False,False,False,False,False,False,True,0.215612,0.212620
1,-0.289776,2,-0.243316,18.0,-0.261338,-0.580100,-0.302803,-0.223475,True,False,...,False,False,False,False,False,False,False,True,0.063588,-0.579552
2,-0.487819,2,-0.212287,14.0,-0.532843,-0.212251,-0.302803,-0.223475,True,False,...,False,True,False,False,False,False,False,True,0.113116,-0.975637
3,-1.379012,2,-0.412563,28.0,-0.903409,-0.212251,-0.302803,-0.223475,False,False,...,False,True,False,False,False,False,False,True,0.372713,-2.758024
4,-1.478033,2,-0.111092,3.0,2.369319,-0.580100,-0.302803,-0.223475,False,False,...,False,False,False,False,False,False,False,True,-0.263212,-2.956066
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
749995,-1.180969,2,0.027479,4.0,2.750893,-0.212251,-0.302803,-0.223475,False,False,...,False,False,False,False,False,False,False,True,0.075591,-2.361938
749996,2.779889,3,-0.202062,19.0,-0.620898,-0.580100,-0.302803,-0.223475,False,False,...,False,False,False,False,False,False,False,True,0.125460,8.339667
749997,0.898482,2,-0.348038,17.0,-0.525505,-0.580100,-0.302803,-0.223475,True,False,...,False,False,False,False,False,False,False,True,0.182895,1.796963
749998,-0.883905,2,-0.521163,26.0,-0.543849,1.259142,-0.302803,-0.223475,False,False,...,False,False,False,False,False,False,False,True,0.283434,-1.767809


In [4]:
y

0         0.0
1         0.0
2         0.0
3         0.0
4         1.0
         ... 
749995    1.0
749996    0.0
749997    0.0
749998    0.0
749999    0.0
Name: y, Length: 750000, dtype: float64

## Define Models

In [7]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier
import joblib

models = {
    'Logistic Regression': LogisticRegression(max_iter=1000),
    'Random Forest': RandomForestClassifier(),
    'XGBoost': XGBClassifier(eval_metric='logloss'),
    "LightGBM": LGBMClassifier(),
    #"SVM": SVC(probability=True),
    "NaiveBayes": GaussianNB(),
    "MLPClassifier": MLPClassifier(hidden_layer_sizes=(64,32), max_iter=500)
}

## Stratified K-Fold Cross-Validation
### Evaluate with ROC-AUC, F1, Precision, Recall, Accuracy

In [8]:
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
results = {}

for name, model in models.items():
    aucs, f1s, precisions, recalls, accs = [], [], [], [], []
    for train_idx, test_idx in skf.split(X, y):
        X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
        y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        y_proba = model.predict_proba(X_test)[:,1] if hasattr(model, "predict_proba") else None
        aucs.append(roc_auc_score(y_test, y_proba) if y_proba is not None else np.nan)
        f1s.append(f1_score(y_test, y_pred))
        precisions.append(precision_score(y_test, y_pred))
        recalls.append(recall_score(y_test, y_pred))
        accs.append(accuracy_score(y_test, y_pred))
    results[name] = {
        'ROC-AUC': np.mean(aucs),
        'F1': np.mean(f1s),
        'Precision': np.mean(precisions),
        'Recall': np.mean(recalls),
        'Accuracy': np.mean(accs)
    }

pd.DataFrame(results).T

[LightGBM] [Info] Number of positive: 72391, number of negative: 527609
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.050136 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1480
[LightGBM] [Info] Number of data points in the train set: 600000, number of used features: 42
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.120652 -> initscore=-1.986273
[LightGBM] [Info] Start training from score -1.986273
[LightGBM] [Info] Number of positive: 72391, number of negative: 527609
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.035940 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1485
[LightGBM] [Info] Number of data points in the train set: 600000, number of used features: 42
[LightGBM] [In

,ROC-AUC,F1,Precision,Recall,Accuracy
Logistic Regression,0.942033,0.594042,0.713824,0.508686,0.916117
Random Forest,0.961660,0.698079,0.754555,0.649478,0.932219
XGBoost,0.965515,0.708772,0.760698,0.663491,0.934217
LightGBM,0.964928,0.707856,0.756077,0.665436,0.933732
NaiveBayes,0.839983,0.457599,0.450974,0.464426,0.867165
MLPClassifier,0.962211,0.703532,0.747108,0.665513,0.932367


## Leaderboard

In [9]:
leaderboard = pd.DataFrame(results).T
leaderboard.sort_values(by='ROC-AUC', ascending=False)
leaderboard

,ROC-AUC,F1,Precision,Recall,Accuracy
Logistic Regression,0.942033,0.594042,0.713824,0.508686,0.916117
Random Forest,0.961660,0.698079,0.754555,0.649478,0.932219
XGBoost,0.965515,0.708772,0.760698,0.663491,0.934217
LightGBM,0.964928,0.707856,0.756077,0.665436,0.933732
NaiveBayes,0.839983,0.457599,0.450974,0.464426,0.867165
MLPClassifier,0.962211,0.703532,0.747108,0.665513,0.932367
